# Parameters

In [ ]:
random_seed = 1
save_to_disk = False
model_name='test_nn'

# Set seed

In [ ]:
from numpy.random import seed
seed(random_seed)
from tensorflow.random import set_seed
set_seed(random_seed)

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier

In [ ]:
from knowledge_distillation.io import load_adult, ASSETS_PATH
from knowledge_distillation.ensemble import UnbiasedAverage
from knowledge_distillation.processing import *
from knowledge_distillation.nn import *

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load data

In [ ]:
df = load_adult()

# Preprocessing

In [ ]:
X, y, target_names = scale_fastai(df, target='salary')

X_train, X_test, y_train, y_test = split_with_seed(X, y)

# Train a simple fully connected NN

In [ ]:
model = KerasClassifier(
    build_fn=create_nn,
    **train_params,
    verbose=1
)

In [ ]:
model.fit(X_train, y_train)

Epoch 1/15
18/18 [==============================] - 4s 109ms/step - loss: 0.6757 - accuracy: 0.6367 - val_loss: 0.5173 - val_accuracy: 0.7607
Epoch 2/15
18/18 [==============================] - 1s 33ms/step - loss: 0.4979 - accuracy: 0.7906 - val_loss: 0.4751 - val_accuracy: 0.7821
Epoch 3/15
18/18 [==============================] - 1s 53ms/step - loss: 0.4425 - accuracy: 0.8209 - val_loss: 0.4659 - val_accuracy: 0.7845
Epoch 4/15
18/18 [==============================] - 1s 51ms/step - loss: 0.4012 - accuracy: 0.8361 - val_loss: 0.4432 - val_accuracy: 0.7907
Epoch 5/15
18/18 [==============================] - 1s 54ms/step - loss: 0.3757 - accuracy: 0.8394 - val_loss: 0.4554 - val_accuracy: 0.7872
Epoch 6/15
18/18 [==============================] - 1s 65ms/step - loss: 0.3589 - accuracy: 0.8446 - val_loss: 0.4249 - val_accuracy: 0.8052
Epoch 7/15
18/18 [==============================] - 1s 56ms/step - loss: 0.3419 - accuracy: 0.8521 - val_loss: 0.4367 - val_accuracy: 0.7967
Epoch 8/15
1

In [ ]:
model.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               3000      
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 200)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5

In [ ]:
# keras.utils.plot_model(model.model, show_shapes=True, dpi=100, rankdir="LR")

# Evaluate single model

In [ ]:
evaluate_model(X_train, X_test, y_train, y_test, model, model_name, save_to_disk=save_to_disk, target_names=target_names)


/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


1/7 [===>..........................] - ETA: 0s

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7/7 [==============================] - 0s 6ms/step
=== Train ===
              precision    recall  f1-score   support

       <=50K       0.85      0.96      0.90     19778
        >50K       0.78      0.47      0.59      6270

    accuracy                           0.84     26048
   macro avg       0.82      0.71      0.74     26048
weighted avg       0.83      0.84      0.83     26048


=== Test ===
              precision    recall  f1-score   support

       <=50K       0.85      0.96      0.90      4942
        >50K       0.78      0.47      0.59      1571

    accuracy                           0.84      6513
   macro avg       0.82      0.71      0.74      6513
weighted avg       0.83      0.84      0.83      6513




,model_name,data,accuracy,precision,recall,f1,auc
0,test_nn,train,0.840602,0.780753,0.469697,0.586537,0.713941
1,test_nn,test,0.840934,0.783669,0.470401,0.587908,0.714561


In [ ]:
z = classification_report(y_train, model.predict(X_train), labels=[0, 1], target_names=target_names)

12/26 [============>.................] - ETA: 0s

/home/jupyterlab/ni-workspace/conda/envs/kd/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


26/26 [==============================] - 0s 8ms/step


# Save model

In [ ]:
if save_to_disk:
    save_keras_classifier(model, model_name)